# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235410487496                   -0.50    8.0
  2   -7.250311187943       -1.83       -1.40    1.0
  3   -7.251290113353       -3.01       -2.09    4.0
  4   -7.251230850463   +   -4.23       -2.12    4.0
  5   -7.251327472566       -4.01       -2.64    3.0
  6   -7.251337531470       -5.00       -3.07    2.0
  7   -7.251338758410       -5.91       -3.92    1.0
  8   -7.251338791172       -7.48       -4.10    4.0
  9   -7.251338797786       -8.18       -4.55    2.0
 10   -7.251338798483       -9.16       -4.86    2.0
 11   -7.251338798691       -9.68       -5.60    3.0
 12   -7.251338798703      -10.91       -5.96    3.0
 13   -7.251338798704      -11.96       -6.40    2.0
 14   -7.251338798705      -13.00       -7.02    3.0
 15   -7.251338798705      -14.57       -7.32    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05154771758349797
[ Info: Arnoldi iteration step 2: normres = 0.6929280304398554
[ Info: Arnoldi iteration step 3: normres = 0.6711810296645497
[ Info: Arnoldi iteration step 4: normres = 0.3257066177882562
[ Info: Arnoldi iteration step 5: normres = 0.32017331782062697
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.34e-02, 7.34e-02, 2.56e-01, 1.77e-01, 2.06e-04)
[ Info: Arnoldi iteration step 6: normres = 0.46351430509586855
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.11e-03, 1.36e-01, 4.05e-01, 1.27e-01, 1.11e-01)
[ Info: Arnoldi iteration step 7: normres = 0.08717574320708643
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.33e-04, 1.83e-02, 2.77e-02, 4.03e-02, 6.03e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10173914764390375
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.03e